<small><i>This notebook was put together by [Abel Meneses](http://www.menesesabad.com) for PyData 2018. Source and license info is on [GitHub](https://github.com/sorice/nlp_pydata2018/).</i></small>

#Etiquetado de las Partes del Discurso#
**Part of Speech Tagging**

[Ver proceso previo: Bases de NLP](#nlp_basic)

**Fecha de elaboración inicial**: agosto de 2015
**Última actualización**: 9 de agosto de 2015
<a id='indice'></a>
##Índice##

1. [Introducción](#introduccion)
    - 1.1 [Ejemplo Treebank en Idioma Inglés.](#ejemplo_treebank_ingles) 
Ejemplo sencillo usando el fragmento del corpus Treebank contenido en NLTK.
    - 1.2 [Conjuntos de Etiquetas.](#tag_set)

2. [Biblioteca NLTK.](#biblioteca_nltk) Usualmente útiles para hacer Minería de Texto.
    - 2.1 [Etiquetado con Unigram](#etiquetador unigram) Etiquetador secuencial.
    - 2.2 [Etiquetado con el Stanford-Tagger](#pos_con_stanfordtagger)
    - 2.3 [Etiquetador configurable ClassifierBasedPOSTagger](#classifierbasedpostagger)

3. [Biblioteca Pattern](#biblioteca_pattern)
    - 3.1 [POS con Parse](#pos_con_parse)

4. [Playfull and Depper Programación NLP](#playfull_programming)
    - 4.1 [Analizador de similaridad usando POS](#pos_similarity)

[Conclusiones](#conclusiones)

[Ejercicios](#ejercicios)

[Referencias](#referencias)

[Índice Alfabético](#indice_alfabetico)

<a id='introduccion'></a>
##Introducción##

El procesado de las partes de las partes del discurso es una técnica para extraer en un
texto las unidades gramaticales como el sintagma nominal sujeto y las partes de la
oración desde el punto de vista léxico, como verbos, etc.

Este compendio contiene ejemplos en español utilizados en 
http://www.nltk.org/book/ch05.html

<a id='ejemplo_treebank_ingles'></a>
## Ejemplo TreeBank Corpus en Inglés

Corpus en inglés para hacer POS Tagging (ver el NLTK Book [Perkins2014](#perkins2014)).

Los ejemplos de oraciones etiquetadas, según el etiquetado del Treebank, pueden ser
encontradas en la carpeta: *nltk\_data/corpora/treebank/tagged* .

Lo que representa cada etiqueta puede ser encontrado en los materiales de este notebook:
[TreeBank Tag Set](files/htmls/3.3/Penn-Treebank_POS_Tagset.html)

In [78]:
from nltk.tag import pos_tag 
from nltk.tokenize import word_tokenize 
string = ''
print('Resultado esperado:')
print('(John, NNP),(s,POS),(big,JJ),(idea,NN),(is,VBZ),(bad,JJ)')
result =  pos_tag(word_tokenize("John's big idea is bad."))
for i in range(len(result)-1):
    string+= '('+result[i][0]+', '+result[i][1]+')'
print('Resultado obtenido:')
print string

Resultado esperado:
(John, NNP),(s,POS),(big,JJ),(idea,NN),(is,VBZ),(bad,JJ)
Resultado obtenido:
(John, NNP)('s, POS)(big, JJ)(idea, NN)(is, VBZ)(bad, JJ)

En este ejemplo pos_tag que está implementado en el *~/nltk/tag/__init__.py*, utiliza
por defecto el modelo del Treebank en inglés. Este fichero es parte del *nltk\_data* y 
es *~/taggers/maxent\_treebank\_pos\_tagger/english.pickle*.

Evidentemente para POS del idioma español este corpus no sirve. Veremos algunas
alternativas en las secciones más adelante.

Otro ejemplo puede ser utilizando el corpus Brown. No olvide copiar el corpus en la 
carpeta que ha definido para los datos, en mi caso ~nltk_data/corpora/ .

In [52]:
from nltk.corpus import brown
brown_tagged_sents = brown.tagged_sents(categories='news')
train_sents = brown_tagged_sents[:30] #cogiendo solo 30 oraciones.
#sacando los tags que se usan aquí:
list = []
for i in range(len(train_sents)):
    for j in range(len(train_sents[i])): 
        if train_sents[i][j][1] not in list:
            list.append(train_sents[i][j][1])

In [54]:
print 'Cantidad de etiquetas usadas',len(list)
print 'tagset:',list[:7],'...'

Cantidad de etiquetas usadas 68
tagset: [u'AT', u'NP-TL', u'NN-TL', u'JJ-TL', u'VBD', u'NR', u'NN'] ...

Como puede verse en el archivo ~/nltk_data/taggers/universal_tagset/en-brown.map este
etiquetado es aún más complejo que el TreeBank. Su lectura podría empezar en 
http://www.hit.uib.no/icame/brown/bcm.html

<a id='tag_set'></a>
# Tag Set

Existen muchos tipos de etiquetas POS, ya hemos mencionado TreeBank y Brown. Cada 
conjunto tiene sus características y su alcance radica en la profundidad del análisis
que se hizo de las oraciones. A continuación exponemos el conjunto llamado universal:

- VERB - verbs (all tenses and modes)
- NOUN - nouns (common and proper)
- PRON - pronouns
- ADJ - adjectives
- ADV - adverbs
- ADP - adpositions (prepositions and postpositions)
- CONJ - conjunctions
- DET - determiners
- NUM - cardinal numbers
- PRT - particles or other function words
- X - other: foreign words, typos, abbreviations
- . - punctuation

Leer más en: http://arxiv.org/abs/1104.2086 
y http://code.google.com/p/universal-pos-tags/

En NLTK 3 existe una función que puede mapear el brown a este tag set universal. Ver
~/nltk/tag/mapping.py
Igualmente en el fichero ~/nltk_data/taggers/universal_tagset/en-brown.map aparece la
conversión del brown al universal.

<a id='Biblioteca NLTK'></a>
# Biblioteca NLTK

<a id='etiquetador unigram'></a>
## Etiquetador Unigram

El siguiente ejemlo utiliza un *corpus etiquetado* en español elaborado en la
Universidad de Barcelona** ver [CESS2007](#cess2007).
La esencia de este ejemplo puede se encontrado en 
[Perkins 2014, *pág 89*](#perkins2014).
Este etiquetador pertenece a las clases programadas para etiquetar oraciones en el 
módulo *nltk/tag/sequential.py*

Los ejemplos de textos
etiquetados pueden encontrarse en la ruta *nltk\_data/corpora/cess\_es*.

In [34]:
from nltk.corpus import cess_esp as cess
from nltk import UnigramTagger as ut
from nltk import BigramTagger as bt

# Read the corpus into a list, 
# each entry in the list is one sentence.
cess_sents = cess.tagged_sents()

# Train the unigram tagger
uni_tag = ut(cess_sents)

In [35]:
import codecs
sentence = "Hoy comí en casa de Lola."

# Tagger reads a list of tokens.
uni_tag.tag(sentence.split(" "))

[('Hoy', 'rg'),
 ('comí', None),
 ('en', 'sps00'),
 ('casa', 'ncfs000'),
 ('de', 'sps00'),
 ('Lola.', None)]

En este primer ejemplo no reconoce el verbo.
El problema fundamental al que se enfrentan los programadores
es que generalmente todo funciona bien para inglés. Cuando intentamos programar algo
para procesar el idioma español, comienzan las dificultades con las tíldes y las ñ.

En los ejemplos, para procesar español, que veremos más adelante resolveremos este
problema. For more details see the next offline [full example](files/htmls/3.3/Dive_Into_NLTK,_Part_III-_Part-Of-Speech_Tagging_and_POS_Tag/index.html)

**Nota:** el corpus *cess* utiliza el etiquetado 
[EAGLES](http://nlp.lsi.upc.edu/freeling/doc/tagsets/tagset-es.html) que es muy
profesional y a la vez complejo. Por ello hemos introducido en los materiales de este
notebook una copia de su explicación para que pueda ser utilizada offline:
[EAGLES Tag Set](files/htmls/3.3/EAGLES_tagset/index.html)

Volver al [*Índice*](#indice).

<a id='pos_con_stanfordtagger'></a>
## POS con Stanford Tagger

[Este ejemplo](files/htmls/3.3/python_-_Stanford_Parser_and_NLTK_-_Stack_Overflow/index.html) fue tomado en su versión original de una 
[URL en Stackoverflow](http://stackoverflow.com/questions/14732465/nltk-tagging-spanish-words-using-a-corpus)

El ejemplo utiliza el etiquetador de la Universidad de Stanford
una de las más avanzadas en el campo de NLP.
</br>**ver etiquetador** http://nlp.stanford.edu/software/tagger.shtml 

También se utiliza el conjunto de etiquetas de la API *FreeLing*, la más avanzada en
el procesamiento del idioma español. Mantenida además por la *Universitat Politècnica 
de Catalunya*
</br>**ver conjunto de etiquetas**: http://nlp.lsi.upc.edu/freeling/doc/tagsets/tagset-es.html or [In here FreeLing-Stanford Tagset](files/htmls/3.3/Freeling-Stanford_Parser_tagset-es.html). Veamos un ejemplo funcional y luego los detalles de como ejecutarlo en tu PC con el menor grado de dificultad.

In [1]:
from nltk.parse.stanford import StanfordParser
from nltk.parse.stanford import StanfordDependencyParser
from nltk.parse.stanford import StanfordNeuralDependencyParser
from nltk.tag.stanford import StanfordPOSTagger, StanfordNERTagger
from nltk.tokenize.stanford import StanfordTokenizer

In [14]:
stanford_pos_dir = '/opt/stanford/stanford-postagger-full-2015-04-20/'
eng_model_filename= stanford_pos_dir + 'models/english-left3words-distsim.tagger'
my_path_to_jar= stanford_pos_dir + 'stanford-postagger.jar'
st = StanfordPOSTagger(model_filename=eng_model_filename, path_to_jar=my_path_to_jar) 
st.tag('What is the airspeed of an unladen swallow ?'.split())

[('What', 'WP'),
 ('is', 'VBZ'),
 ('the', 'DT'),
 ('airspeed', 'NN'),
 ('of', 'IN'),
 ('an', 'DT'),
 ('unladen', 'JJ'),
 ('swallow', 'VB'),
 ('?', '.')]

**Nota:** Un detalle importante es que las oraciones que se etiquetan deben tener los signos de puntuación separados. Como se verá más adelante Pattern sí los identifica.

In [18]:
es_model_filename= stanford_pos_dir + 'models/spanish.tagger'
spanish_postagger = StanfordPOSTagger(model_filename=es_model_filename, path_to_jar=my_path_to_jar, encoding='utf8')

sentences = ['El copal se usa principalmente para sahumar en distintas'
             + 'ocasiones como lo son las fiestas religiosas.','Las flores, '
             + 'hojas y frutos se usan para aliviar la tos y también se emplea'
             + 'como sedante.']

sentences = ['Hoy comí en casa de Lola.']

spanish_postagger.tag(sentences)

[('Hoy', 'rg'),
 ('comí', 'vmis000'),
 ('en', 'sp000'),
 ('casa', 'nc0s000'),
 ('de', 'sp000'),
 ('Lola.', 'np00000')]

In [10]:
print(sentences)

['El copal se usa principalmente para sahumar en distintasocasiones como lo son las fiestas religiosas.', 'Las flores, hojas y frutos se usan para aliviar la tos y también se empleacomo sedante.']


In [19]:
for sent in sentences:
    tagged_words = spanish_postagger.tag(sent.split())
    #print (tagged_words)
    nouns = []

    for (word, tag) in tagged_words:
        #print(word+' '+tag)
        if tag.startswith('n'): nouns.append(word)

    print(nouns)

['casa', 'Lola.']


Copie el Stanford-Postager.jar en la carpeta donde está realizando sus experimentos.
Adicione una carpeta *models* y dentro el modelo del lenguaje que va a parsear.

(opcional) Algunas soluciones utilizan directamente los modelos, para ello descompacte el .jar de los modelos, y luego colóquelos en su carpeta de trabajo STANFORD_MODELS_DIR:
<html>
<br><blockquote>\$> unzip stanford-corenlp-3.7.0-models.jar<br>
\$> mv edu/stanford/nlp/models/ STANFORD_MODELS_DIR</blockquote>
</html>

Se debe tener cuidado con las versiones de ambos proyectos (NLTK y el Stanford-Tagger o
Stanford-Parser) pues como son proyectos en constante desarrollo cambian rapidamente.
Este notebook está configurado para usar los sigtes paquetes

* http://nlp.stanford.edu/software/stanford-ner-2015-04-20.zip
* http://nlp.stanford.edu/software/stanford-postagger-full-2015-04-20.zip
* http://nlp.stanford.edu/software/stanford-parser-full-2015-04-20.zip
* versión 3.1 de NLTK https://pypi.python.org/pypi/nltk

**Notes:** 

<html><ul>
<li>En Ubuntu GNU/Linux se recomienda usar a partir del LTS 16.04 pues es stanford-tools ha sido desarrollada para para la la versión 1.8 de java del openJDK pues esta versión del   esta versión de java o superior. En Windows utilizar Python3.4 así como las versiones de 32bits pues hay algunos bugs importantes en la versión de 64bits (al menos hasta el 15 de enero de 2016).

<li>Al final si se intenta con java 1.7, nltk 3.0.3 y el stanford-tagger-2015-04-20 da este error al ejecutar "javac -cp stanford-tagger.jar TaggerDemo.java":<br>

<p><code>*TaggerDemo.java:6: cannot access edu.stanford.nlp.ling.Sentence... 
class file has wrong version 52.0, should be 50.0*</code>
</html>

Volver al [*Índice*](#indice).

<a id='classifierbasedpostagger'></a>
## Etiquetador configurable ClassifierBasedPOSTagger de NLTK

Este etiquetador es parametrizable con clasificadores que ya tenemos. Inicialmente
podemos decir que devuelve las palabras etiquetadas con POS(no sabría decir aún)

In [11]:
from nltk.corpus import treebank
train_sents = treebank.tagged_sents()[:50]
test_sents = treebank.tagged_sents()[51:101]

from nltk.tag.sequential import ClassifierBasedPOSTagger
from nltk.classify import MaxentClassifier
me_tagger = ClassifierBasedPOSTagger(train=train_sents,classifier_builder=MaxentClassifier.train)
me_tagger.evaluate(test_sents)

  ==> Training (100 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -3.55535        0.006
             2          -0.90550        0.970
             3          -0.55974        0.999
             4          -0.40216        1.000
             5          -0.31346        1.000
             6          -0.25672        1.000
             7          -0.21736        1.000
             8          -0.18847        1.000
             9          -0.16636        1.000
      Training stopped: keyboard interrupt
         Final          -0.16636        1.000

0.7685025817555938

**OJO:** Si la corrida demora, apriete el botón interrupt del notebook.

In [12]:
me_tagger.tag(['I','run','for','the','beach'])

[('I', u'DT'),
 ('run', u'NN'),
 ('for', u'IN'),
 ('the', u'DT'),
 ('beach', u'NN')]

Como se pude observar, para idioma inglés, este método etiqueta correctamente el 76% de
los casos. En el libro se documenta que tras probar con unas 3000 oraciones la
eficiencia de este algoritmo rebasa el 93%. Aquí solo se ha entrenado con 50 oraciones
por problemas de rendimiento.

Volver al [*Índice*](#indice).

<a id='biblioteca_pattern'></a>
## Biblioteca Pattern

<a id='pos_con_parse'></a>
## POS con Parse

El cuarto ejemplo está dedicado a introducir más formalmente la biblioteca belga
**Pattern**. En este caso su módulo para hacer POS en idioma español. Como pudo verse
en el ejemplo en inglés vs español hecho con NLTK, la implementación de estas funciones
para ambos idiomas son diferentes. Sin embargo en *Pattern* la implementación es 
similar.

La ayuda para esta biblioteca puede ser encontrada online en 
[online Pattern](http://www.clips.ua.ac.be/pages/pattern).

**Note:** Pattern library is only available on Python2. Any further problem with this section must be corrected running the notebook on Ipython2.

In [49]:
from pattern.en import parse, pprint
en = 'Jonh saw the fish'
en = parse(en, relations=True, lemmata=True)
pprint(en)

          WORD   TAG    CHUNK   ROLE   ID     PNP    LEMMA   
                                                             
          Jonh   NNP    NP      SBJ    1      -      jonh    
           saw   VBD    VP      -      1      -      saw     
           the   DT     NP      OBJ    1      -      the     
          fish   NN     NP ^    OBJ    1      -      fish    

In [15]:
from pattern.es import parse, pprint
es = 'El estudiante comió el pescado de su profesor demasiado tarde.'
es = parse(es, relations=True, lemmata=True)
pprint(es)

          WORD   TAG    CHUNK   ROLE   ID     PNP    LEMMA        
                                                                  
            El   DT     NP      SBJ    1      -      el           
    estudiante   NN     NP ^    SBJ    1      -      estudiante   
         comió   VB     VP      -      1      -      comer        
            el   DT     NP      OBJ    1      -      el           
       pescado   NN     NP ^    OBJ    1      -      pescado      
            de   IN     PP      -      -      PNP    de           
            su   PRP$   NP      -      -      PNP    su           
      profesor   NN     NP ^    -      -      PNP    profesor     
     demasiado   RB     NP ^    -      -      PNP    demasiado    
         tarde   RB     NP ^    -      -      PNP    tarde        
             .   .      -       -      -      -      .            

In [16]:
type(es)

en.parser.TaggedString

Como se puede observar el objeto *"es"* es del tipo *TaggedString*. 

Cuando se revisa
el código del *pattern/text/es/parser/\_\_init\_\_.py* se puede comprobar que el 
**Parole Tag** es convertido a **Treebank Tag**. La presición de este algoritmo de
pattern es de alrededor del 92% según el propio fichero, pero la conversión de Parole a
Treebank puede incurrir en problemas adicionales.

**Manipulemos el objeto "es"**

In [47]:
test=es.split()
newtest = []

for i in test[0]:
    newtest.append(i)

print newtest[2]
print test[0][2][0].encode('utf8')+" -> "+test[0][0][1].encode('utf8')

[u'comi\xf3', u'VB', u'B-VP', u'O', u'VP-1', u'comer']
comió -> DT

Volver al [*Índice*](#indice).

<a id='conclusiones'></a>

## Conclusiones
- El POS...

Volver al [*Índice*](#indice).

<a id='ejercicios'></a>

## Ejercicios

* **Ejercicio 1:** Implemente un POS en español utilizando un clasificador...

Volver al [*Índice*](#indice).

<a id='references'></a>
## Referencias

* <a id='perkins2014'></a>**[Perkins2014]** Jacob Perkins, *Python 3 Text Processing with NLTK 3 Cookbook*, 2014. 
Capítulo 5 *Part-of-speech Tagging*

* <a id='cess2007'></a>**[CESS2007]** Antonia Martí, Mariona Taulé, Lluís Márquez, Manuel Bertran. 2007. 
CESS-ECE: A Multilingual and Multilevel Annotated Corpus. 
*see* http://www.lsi.upc.edu/~mbertran/cess-ece/publications

* **NLTK** Bird, Steven, Edward Loper and Ewan Klein (2009), Natural 
Language Processing with Python. O’Reilly Media Inc. *see* http://nltk.org/

* (colocar referencias al Pen Treebank POS Tags Set)

<a id='alphabetic_index'></a>
## Índice Alfabético

<a id='token'></a>
**Token**: señal, indicio, muestra. Se usa generalmente para referirse a la unidad
más pequeña de procesamiento: palabras, fonemas, n-grams, etc..


Volver al [*Índice*](#indice).